In [1]:
import pandas as pd
import numpy as np
import functools as ft
import requests
from bs4 import BeautifulSoup
import utm

import sys

sys.path.append("..")
from tools import data_path, output_path

### Data origin: 
Data was privately provided by berlin authorities. Not reproducible without original data.

In [2]:
f = open(data_path + "WQ_Blunk/check.txt", "r", errors="replace")
data = f.read()
data = data.split("\n")
data = [x.split(" ") for x in data]

In [3]:
stack = []
current = []
meta = [data[2][0].split("/")[2]]
for x in data[7:]:
    if len(x) != 2:
        if len(current) == 0:
            if "Senat" in x[0]:
                meta.append(x[0].split("/")[2])
        else:
            stack.append(current)
            current = []
    else:
        current.append(x)

In [4]:
merge_stack = []
for n, x in enumerate(stack):
    parse = pd.DataFrame(x)
    parse[0] = pd.to_datetime(parse[0])
    parse.loc[parse[1] == "-777", 1] = np.nan
    parse[1] = parse[1].astype(float)
    parse.columns = ["datetime", meta[n]]
    merge_stack.append(parse)

In [5]:
stack = ft.reduce(
    lambda left, right: pd.merge(left, right, on="datetime", how="outer"), merge_stack
)
stack.index = stack["datetime"]
stack.drop(columns="datetime", inplace=True)
stack.columns = [x + "_b" for x in stack.columns]

In [18]:
# Get meta data.
info_stack = []
for x in stack.columns[1:]:
    print(x)
    response = requests.get(
        "https://wasserportal.berlin.de/station.php?anzeige=i&thema=ows&station="
        + str(x[:-2])
    )
    soup = BeautifulSoup(response.text, "html.parser")
    indiatable = soup.find("table", {"class": "wikitable"})
    infos = (
        soup.text.split(
           "Sauerstoffgehalt\nSauerstoffsättigung\nKennwerte\nLagekarte\n\n\n\n\n \n\n\n\n\n\n"
        )[1]
        .split("Blättern durch alle Berliner Messstellen:")[0]
        .split("\n")
    )

    info_stack.append("".join([x for x in infos if len(x) > 50]))

5865300_b
5867900_b
5827101_b
5870100_b
5826701_b
5862811_b
5827700_b
5803200_b
5866301_b


In [24]:
table = []
for info in info_stack:
    current = info
    splitters = [
        "Nummer",
        "Name",
        "Gewässer",
        "Betreiber",
        "Ausprägung",
        "Flusskilometer",
        "Pegelnullpunkt (m +NHN)",
        "Rechtswert (UTM 33 N)",
        "Hochwert (UTM 33 N)",
    ]
    info_hold = []
    for splitter in splitters:

        sep = current.split(splitter)
        info_hold.append(sep[0])
        current = sep[1]
    info_hold.append(current)
    table.append(info_hold)

In [26]:
table = pd.DataFrame(table, columns=[" "] + splitters)
table.index = table["Nummer"]
table.drop(columns="Nummer", inplace=True)

In [ ]:
table["Hochwert (UTM 33 N)"] = [x[0] for x in table["Hochwert (UTM 33 N)"].str.split("An")]

In [35]:
table["Rechtswert (UTM 33 N)"] = table["Rechtswert (UTM 33 N)"].astype(int)
table["Hochwert (UTM 33 N)"] = table["Hochwert (UTM 33 N)"].astype(int)

### Bring into general Format

In [36]:
# berlin parsing
for ind, line in table.iterrows():
    table.loc[ind, ["Rechtswert (UTM 33 N)", "Hochwert (UTM 33 N)"]] = utm.to_latlon(
        line["Rechtswert (UTM 33 N)"], line["Hochwert (UTM 33 N)"], 33, "N"
    )

/tmp/ipykernel_4182074/3168747043.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '52.45907678803533' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  table.loc[ind, ["Rechtswert (UTM 33 N)", "Hochwert (UTM 33 N)"]] = utm.to_latlon(
/tmp/ipykernel_4182074/3168747043.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13.576348303220911' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  table.loc[ind, ["Rechtswert (UTM 33 N)", "Hochwert (UTM 33 N)"]] = utm.to_latlon(


In [37]:
# Flusskilometer seems to be distance to end according to maps check.

In [38]:
table = table[
    [
        "Gewässer",
        "Rechtswert (UTM 33 N)",
        "Hochwert (UTM 33 N)",
        "Flusskilometer",
        "Pegelnullpunkt (m +NHN)",
    ]
]
table.columns = ["R", "X", "Y", "D", "H"]
table.index.name = "ID"

In [39]:
table["QD"] = -1
table["QH"] = -1
table["QX"] = -1
table["QY"] = -1
table["QR"] = -1

In [40]:
table.loc[~(table["X"].isnull()), "QX"] = 0
table.loc[~(table["Y"].isnull()), "QY"] = 0
table.loc[~(table["D"].isnull()), "QD"] = 0
table.loc[~(table["R"].isnull()), "QR"] = 0
table.loc[~(table["H"].isnull()), "QH"] = 0

In [41]:
table.to_csv(output_path + "berlin_meta_data.csv")

In [42]:
stack.to_csv(output_path + "/berlin_processed.csv")